In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Aluna: Luciana Yared
Matrícula: 2086324017

In [ ]:
#carregar os dados
treino = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
teste = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

In [ ]:
treino.shape, teste.shape

In [ ]:
#juntar os dataframes
df = treino.append(teste)
df.head(5)

In [ ]:
df.info()

In [ ]:
#remover a coluna Unnamed:0
del df['Unnamed: 0']

In [ ]:
#remover a coluna id
del df['id']

In [ ]:
df.shape

In [ ]:
df['satisfaction'].value_counts()

In [ ]:
#verificar valores nulos
df.isna().sum()

In [ ]:
#transformar os valores nulos em 0
df.fillna(0, inplace=True)

In [ ]:
df.isna().sum()

# Análise Exploratória

In [ ]:


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")

corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))

sns.heatmap(corr, mask=mask, cmap="Spectral", vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})



In [ ]:
corr

In [ ]:
sns.barplot(
data = df, 
x = "Flight Distance", 
y = "satisfaction",
palette = "magma")

In [ ]:
sns.histplot(
data = df,  
x = "Age",
hue = "satisfaction",
palette = "viridis")

In [ ]:
df.describe()

In [ ]:
sns.barplot(data=df, 
                 x="Age",
                 y="satisfaction",
                 hue = "Class",
                 palette = "magma"
                )

In [ ]:
sns.barplot(data=df, 
                 x="Departure/Arrival time convenient",
                 y="satisfaction",
                 hue = "Class",
                 palette = "magma"
                )

In [ ]:
sns.barplot(data=df, 
                 x="Flight Distance",
                 y="satisfaction", 
                 hue = "Class",
                 palette = "magma"
                )

In [ ]:
sns.barplot(data=df, 
                 x="Seat comfort",
                 y="satisfaction", 
                 hue = "Class",
                 palette = "magma"
                )

# Modelo

In [ ]:
# transformação os valores da coluna target em numéricos
df['satisfaction'] = df['satisfaction'].replace({'satisfied': 1, 'neutral or dissatisfied' : 0}).astype(int)

In [ ]:
#verificar quais colunas são do tipo object
df.select_dtypes('object').head()

In [ ]:
#criação de dummies para as variáveis categóricas
df2 = pd.get_dummies(df, columns=['Gender', 'Customer Type', 'Class',
                                 'Type of Travel'])
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.info()

In [ ]:
from sklearn.model_selection import train_test_split

#Particionar o dataframe

#treino e teste
train, test = train_test_split(df2, test_size=0.20, random_state=42)

#treino e validação
train, valid = train_test_split(train, test_size=0.20, random_state=42)

train.shape, valid.shape, test.shape

In [ ]:
# Colunas de entrada:
entrada = [c for c in df2.columns if c not in ['satisfaction']]

entrada

# Treinamento de modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
# treinar o modelo
rf.fit(train[entrada], train['satisfaction'])

In [ ]:
# predição
pred = rf.predict(valid[entrada])

pred

## Avaliação de desempenho

In [ ]:
from sklearn.metrics import accuracy_score
# Precisão da previsão nos dados de validação
accuracy_score(valid['satisfaction'], pred)

In [ ]:
# Precisão da previsão nos dados de teste
pred_test = rf.predict(test[entrada])

accuracy_score(test['satisfaction'], pred_test)

In [ ]:
df['satisfaction'].value_counts(normalize=True)